# 특성 중요도 조사

모델 성능을 분석하는 한 가지 좋은 방법은 예측을 만드는데 어떤 특성이 가장 많이 사용되고 어떤 특성이 전혀 도움이 되지 않는지 조사하는 것입니다. 이를 특성 중요도 분석이라고 합니다.

먼저 데이터를 로드합니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import joblib

import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')

from ml_editor.data_processing import (
    format_raw_df,
    get_split_by_author,  
    add_text_features_to_df,
    get_vectorized_series, 
    get_feature_vector_and_label
)
from ml_editor.model_evaluation import get_feature_importance

data_path = Path('../data/writers.csv')
df = pd.read_csv(data_path)
df = format_raw_df(df.copy())

그다음 특성을 추가하고 데이터를 나눕니다.

In [2]:
df = add_text_features_to_df(df.loc[df["is_question"]].copy())
train_df, test_df = get_split_by_author(df, test_size=0.2, random_state=40)

사전 훈련된 모델과 벡터화 객체를 로드합니다.

In [3]:
model_path = Path("../models/model_1.pkl")
clf = joblib.load(model_path) 
vectorizer_path = Path("../models/vectorizer_1.pkl")
vectorizer = joblib.load(vectorizer_path) 

텍스트를 벡터로 변환하고 모델을 위해 특성을 준비합니다.

In [4]:
train_df["vectors"] = get_vectorized_series(train_df["full_text"].copy(), vectorizer)
test_df["vectors"] = get_vectorized_series(test_df["full_text"].copy(), vectorizer)

features = [
                "action_verb_full",
                "question_mark_full",
                "text_len",
                "language_question",
            ]
X_train, y_train = get_feature_vector_and_label(train_df, features)
X_test, y_test = get_feature_vector_and_label(test_df, features)

이제 사이킷런 API를 사용해 가장 중요한 특성과 그렇지 않은 특성을 출력합니다.

In [5]:
w_indices = vectorizer.get_feature_names()
w_indices.extend(features)
all_feature_names = np.array(w_indices)

In [6]:
k = 10
print("상위 %s개 중요도:\n" % k)
print('\n'.join(["%s: %.2g" % (tup[0], tup[1]) for tup in get_feature_importance(clf, all_feature_names)[:k]]))

print("\n하위 %s개 중요도:\n" % k)
print('\n'.join(["%s: %.2g" % (tup[0], tup[1]) for tup in get_feature_importance(clf, all_feature_names)[-k:]]))

상위 10개 중요도:

text_len: 0.01
are: 0.0059
what: 0.0058
writing: 0.0051
can: 0.0044
ve: 0.0044
on: 0.0039
do: 0.0038
some: 0.0037
story: 0.0037

하위 10개 중요도:

toes: 0
anyhow: 0
waking: 0
winning: 0
nerves: 0
fifty: 0
newcomer: 0
nightmares: 0
nineteen: 0
instrument: 0


텍스트 길이가 가장 중요한 특성임을 볼 수 있습니다. 그다음 중요한 특성은 자주 사용되는 영어 단어입니다. 유용한 글쓰기 추천을 제공하기 위해 모델을 사용하려면 행동으로 옮길 수 있는 글쓰기 추천으로 쉽게 변환할 수 있는 특성을 만들어야 합니다.